In [51]:
%%writefile mapper1.py
#! /usr/bin/python
# Your code for mapper here.
import sys
import re

path = 'stop_words_en.txt'

# Your code for reading stop words here
with open(path, "r") as f:
    stop_words = f.read().splitlines()
reload(sys) 
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        words = re.split("\W*\s+\W*", text.lower(), flags=re.UNICODE)
        n=0
        word_dic={}
        for word in words:
            #word_l=word.lower()
            if word not in stop_words:
                n+=1
                if word_dic.has_key(word):
                    word_dic[word]=word_dic[word]+1
                else:
                    word_dic[word]=1
        for word,nt in word_dic.items():
            tf=float(nt)/float(n)
            print("{}\t{}\t{}\t{}".format(word,article_id, tf,1))
                #print "%s\t%s\t%d\t%d" % (word,article_id, tf,1)

    except Exception as e:
        print(e)
        continue

Overwriting mapper1.py


In [52]:
%%writefile reducer1.py

import sys
from math import log
current_count = 0
current_word = None
dic_article={}
for line in sys.stdin:

    try:
        word,article_id,tf,count  = line.strip().split('\t', 3)
        count = int(count)
        tf = float(tf)
        if current_word != word:  
            if current_word:
                for article,term_f in dic_article.items():
                    idf = float(1)/log(1+current_count)
                    print "%s\t%s\t%f" % (current_word,article,term_f*idf)
            current_word = word
            current_count = 0
            dic_article.clear()
            
        dic_article[article_id]=tf        
        current_count += count
        
    except ValueError as e:
        continue
        
if current_word:   
    for article,term_f in dic_article.items():
        idf = 1/log(1+current_count)
        print "%s\t%s\t%f" % (current_word,article,term_f*idf)



Overwriting reducer1.py


In [27]:
%%bash

OUT_DIR="task_2_1_"$(date +"%s%6N")
NUM_REDUCERS=4

hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="TFIDF" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null


# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR}/part-*| grep -w "labor" | grep -w "12" | cut -f 3

rm: `mapper1.py': No such file or directory
rm: `README.md': No such file or directory
rm: `reducer1.py': No such file or directory
rm: `StopWordsTask2.ipynb': No such file or directory
rm: `supervisord.log': No such file or directory
rm: `supervisord.pid': No such file or directory
rm: `Week6Task2_TFIDF.ipynb': No such file or directory
rm: `WordCountTask0.ipynb': No such file or directory
rm: `WordsRatingTask1.ipynb': No such file or directory
19/09/15 23:13:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/15 23:13:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/15 23:14:00 INFO mapred.FileInputFormat: Total input files to process : 1
19/09/15 23:14:00 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	